In [1]:
# Import necessary libraries
import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, extract_model_features, compute_knn_indices, compute_overlap_matrix, plot_overlap_matrix, split_shuffle_data

In [2]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/luna.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [3]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

In [4]:
# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features

    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue

    # Extract paths and labels for train, val, and test sets
    train_labels = [v["row"]["malignancy"] for v in values["train"]] # Extract malignancy labels for the training set
    val_labels = [v["row"]["malignancy"] for v in values["val"]] # Extract malignancy labels for the validation set
    test_labels = [v["row"]["malignancy"] for v in values["test"]] # Extract malignancy labels for the test set
    
    # Stack features
    train_items = np.vstack([v["feature"] for v in values["train"]]) # Stack features for the training set
    val_items = np.vstack([v["feature"] for v in values["val"]]) # Stack features for the validation set
    test_items = np.vstack([v["feature"] for v in values["test"]]) # Stack features for the test set



    # Train model with hyperparameter optimization
    best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = evaluate_model(best_model, test_items, test_labels)

[I 2025-02-27 14:50:35,007] A new study created in memory with name: no-name-6fac199e-9e28-4371-ab4d-347d483efb86
[I 2025-02-27 14:50:35,022] Trial 0 finished with value: 0.8810625175217268 and parameters: {'k': 29}. Best is trial 0 with value: 0.8810625175217268.
[I 2025-02-27 14:50:35,034] Trial 1 finished with value: 0.8814830389683207 and parameters: {'k': 12}. Best is trial 1 with value: 0.8814830389683207.
[I 2025-02-27 14:50:35,043] Trial 2 finished with value: 0.8832352116624614 and parameters: {'k': 11}. Best is trial 2 with value: 0.8832352116624614.
[I 2025-02-27 14:50:35,053] Trial 3 finished with value: 0.8796607793664144 and parameters: {'k': 42}. Best is trial 2 with value: 0.8832352116624614.
[I 2025-02-27 14:50:35,063] Trial 4 finished with value: 0.8395710681244744 and parameters: {'k': 3}. Best is trial 2 with value: 0.8832352116624614.
[I 2025-02-27 14:50:35,073] Trial 5 finished with value: 0.88085225679843 and parameters: {'k': 28}. Best is trial 2 with value: 0.8

In [ ]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict, font_size=16, height=600, width=600, marker_color="#ADD8E6")
fig.show() # Show the plot

In [14]:
model_features = extract_model_features(data)
model_neighbors = compute_knn_indices(model_features, num_neighbors=10, metric="cosine")
overlap_matrix, model_list = compute_overlap_matrix(model_neighbors)
fig = plot_overlap_matrix(overlap_matrix, model_list)
fig.show()